# Proportion of working-age people in ACS not employed, and employed by military

In [1]:
import pandas as pd, numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from numpy.random import default_rng
import scipy.stats as stats

pd.set_option('display.min_rows', 20)

! whoami
! date

zmbc
Tue Nov 29 13:18:42 PST 2022


## Read PUMS

PUMS data dictionary: https://www2.census.gov/programs-surveys/acs/tech_docs/pums/data_dict/PUMS_Data_Dictionary_2016-2020.pdf

In [2]:
acs = pd.read_hdf('../data/acs_2020_5yr_person.hdf', key='acs')

In [3]:
# Duplicate indices! In the future, should probably deal with this in download_acs!
# Filter to relevant columns to save memory
acs = acs[['SERIALNO', 'HISP', 'RAC1P', 'AGEP', 'SEX', 'ESR', 'RELSHIPP', 'PWGTP']].reset_index(drop=True)

### Define employment

ESR meaning (see data dictionary):

1. Civilian employed, at work
2. Civilian employed, with a job but not at work
3. Unemployed
4. Armed forces, at work
5. Armed forces, with a job but not at work
6. Not in labor force

"Not at work" means that they had a job but hadn't worked in the previous week, e.g. were on vacation.

In [4]:
acs['working_age'] = (acs.AGEP >= 18)
acs['employed'] = np.where(acs.ESR.isnull(), np.nan, (acs.ESR.isin([1, 2, 4, 5])).astype(int))

## Calculate proportion employed

In [5]:
employed_proportion = acs[(acs.working_age) & (acs.employed == 1)].PWGTP.sum() / acs[(acs.working_age)].PWGTP.sum()
employed_proportion

0.6114367314410635

In [6]:
1 - employed_proportion

0.3885632685589365

In [7]:
print(f'{1 - employed_proportion:.2%}')

38.86%


## Calculate proportion employed by military

In [8]:
acs['military_employed'] = np.where(acs.ESR.isnull(), np.nan, (acs.ESR.isin([4, 5])).astype(int))

In [9]:
# Total size of military: does this make sense? A quick Google says yes.
acs[(acs.working_age) & (acs.military_employed == 1)].PWGTP.sum()

1132832

In [10]:
military_employed_overall = acs[(acs.working_age) & (acs.military_employed == 1)].PWGTP.sum() / acs[(acs.working_age)].PWGTP.sum()
military_employed_overall

0.004472354972757207

In [11]:
print(f'{military_employed_overall:.2%}')

0.45%


RELSHIPP 38 is noninstitutionalized group quarters population, which would be everyone living in military GQ.

In [12]:
military_employed_non_gq = acs[(acs.working_age) & (acs.RELSHIPP != 38) & (acs.military_employed == 1)].PWGTP.sum() / acs[(acs.working_age)].PWGTP.sum()
military_employed_non_gq

0.0032002412224113722

In [13]:
print(f'{military_employed_non_gq:.2%}')

0.32%
